In [1]:
import pandas as pd
import requests
import selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from bs4 import BeautifulSoup
from time import sleep 
import re
import sys 
import os
sys.path.append(os.path.abspath("../src"))   
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns


import funciones as f

In [2]:
df_total=pd.read_csv("../datos/datos_Supers_completo.csv")

In [3]:
df_total.head()

,Día,Precio (€),Variación,supermercado,producto,variante
0,2024-07-12,21.4,0.0,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
1,2024-07-13,21.4,0.0,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
2,2024-07-14,21.4,0.0,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
3,2024-07-15,21.4,0.0,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l
4,2024-07-16,21.4,0.0,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l


In [4]:
df_total["Día"]=pd.to_datetime(df_total["Día"],format= "mixed", dayfirst= True)

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132954 entries, 0 to 132953
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Día           132954 non-null  datetime64[ns]
 1   Precio (€)    132954 non-null  float64       
 2   Variación     132954 non-null  float64       
 3   supermercado  132954 non-null  object        
 4   producto      132954 non-null  object        
 5   variante      132954 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 6.1+ MB


In [6]:
df_total.describe().T

,count,mean,min,25%,50%,75%,max,std
Día,132954,2024-08-30 11:24:23.946929152,2024-06-22 00:00:00,2024-08-04 00:00:00,2024-08-29 00:00:00,2024-09-25 00:00:00,2024-10-26 00:00:00,NaN
Precio (€),132954.0,9.734827,0.5,1.41,5.22,11.19,154.32,13.650662
Variación,132954.0,-0.003576,-21.33,0.0,0.0,0.0,20.69,0.53269


In [7]:
df_total.describe(include="O").T

,count,unique,top,freq
supermercado,132954,6,hipercor,41096
producto,132954,3,leche,68599
variante,132954,1582,aceite-de-oliva-virgen-extra-picual-casa-junca...,242


COMENTARIOS EDA inicial:  



- La media de precio de los principales supermercados (en los productos de aceite de girasol, aceite de oliva y leche) en España es de 9,73€, mientras que la mediana es de 5,22€ indicando que en los principales supermercados tienen ciertos productos bastante más caros mientras que parece haber una mayor cantidad de productos "baratos" (pues para calcular la mediana se ordenann de menor a mayor los productos más baratos están al principio).

- En cuanto a la variación de precio, se observa que por lo general no hay variaciones y son bastante estables sin enbargo si que comprobamos un máximo y un mínimo de variación muy destacables(lo que podría indicar o bien promociones (como el descuento del IVA) o/y subidas de precio abusivas)

Otros datos de interés:  

- El supermercado que más aparece es Hipercor también el producto Leche y la variante Aceite de oliva virgen extra Picual Casa Juncal.	


**CREACION DE BASE DE DATOS EN DBEAVER**

Primero diseño los Data Frames que serán mis futuras tablas en la la base de datos

In [8]:
df_supermercados= pd.DataFrame()
df_supermercados["super"]= df_total["supermercado"].unique()

df_productos= pd.DataFrame()
df_productos["producto"]= df_total["producto"].unique()

In [9]:
df_diccsupe=f.crear_df_id (df_supermercados, "super")


In [10]:
df_diccpro=f.crear_df_id(df_productos, "producto")

In [11]:
df_principal= pd.DataFrame()
df_principal["fecha"]= df_total["Día"]
df_principal["supermercado"]= df_total["supermercado"]
df_principal["producto"]= df_total["producto"]
df_principal["variante"]=df_total["variante"]
df_principal["precio"]= df_total["Precio (€)"]
df_principal["variacion"]=df_total["Variación"]
df_principal.head()

,fecha,supermercado,producto,variante,precio,variacion
0,2024-07-12,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
1,2024-07-13,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
2,2024-07-14,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
3,2024-07-15,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0
4,2024-07-16,carrefour,aceite-de-girasol,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0


In [12]:
f.añadir_id_mapeado(df_diccpro,"producto","id_producto","producto","id_producto", df_principal).sample(3)

,fecha,supermercado,variante,precio,variacion,id_producto
5869,2024-10-18,carrefour,aceite-de-oliva-virgen-extra-de-nuestra-tierra...,30.80,0.00,2
19009,2024-07-04,mercadona,aceite-de-oliva-virgen-extra-hacendado-02-l,3.14,0.00,2
56883,2024-10-01,dia,leche-semidesnatada-calcio-pascual-movit-brik-1-l,3.05,0.06,3


In [13]:
f.añadir_id_mapeado(df_diccsupe,"super","id_super","supermercado","id_supermercado", df_principal).sample(3)

,fecha,variante,precio,variacion,id_producto,id_supermercado
128793,2024-08-20,président-leche-desnatada-de-vaca-de-origen-10...,1.45,0.0,3,6
63058,2024-10-22,borges-aceite-de-oliva-intenso-08-garrafa-5-l,56.92,0.0,2,5
52478,2024-08-15,leche-desnatada-lauki-brik-1-l,9.54,0.0,3,4


In [14]:
df_principal.index = pd.RangeIndex(start=1,stop = len(df_principal)+1,step=1)
df_principal["id_principal"]=df_principal.index


In [15]:
df_principal.head(2)

,fecha,variante,precio,variacion,id_producto,id_supermercado,id_principal
1,2024-07-12,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0,1,1,1
2,2024-07-13,aceite-de-girasol-capicua-garrafa-5-l,21.4,0.0,1,1,2


In [16]:
df_principal.sample(5)

,fecha,variante,precio,variacion,id_producto,id_supermercado,id_principal
91631,2024-10-15,lacturale-leche-semidesnatada-brik-1-l,1.26,0.0,3,5,91631
42286,2024-09-13,leche-semidesnatada-calcio-pascual-brik-1-litro,1.75,0.0,3,3,42286
79016,2024-08-01,oro-bailen-aceite-de-oliva-virgen-extra-picual...,13.29,0.0,2,5,79016
85585,2024-08-31,covap-leche-desnatada-sin-lactosa-brik-1-l,1.39,0.0,3,5,85585
131921,2024-10-09,puleva-leche-semidesnatada-de-vacas-con-alimen...,6.54,0.0,3,6,131921


Ahora conecto con mi base de datos para empezar a crear allí las tablas con los datos

In [17]:
try:
    conexion= psycopg2.connect(
        database= "Precios_productos_basicos",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

TABLA Supers

In [18]:
cursor= conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS mercados(
    super VARCHAR(100),
    id_super INT PRIMARY KEY   
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [19]:
lista_de_tuplas=[]
for fila in df_diccsupe.values:
    lista_de_tuplas.append(tuple(fila))


In [20]:
     
query_insercion= "INSERT INTO mercados (super, id_super) VALUES (%s, %s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

TABLA PRODUCTOS

In [21]:
cursor=conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS productos(
    producto VARCHAR(100),
    id_producto INT PRIMARY KEY
);
""" 
cursor.execute(query_creacion)
conexion.commit()

In [22]:
lista_de_tuplas= []
for fila in df_diccpro.values:
    lista_de_tuplas.append(tuple(fila))

In [23]:
query_insercion= "INSERT INTO productos(producto, id_producto)  VALUES (%s,%s);"
cursor.executemany(query_insercion, lista_de_tuplas)
conexion.commit()

TABLA PRINCIPAL

In [24]:
cursor=conexion.cursor()
query_creacion= """
CREATE TABLE IF NOT EXISTS principal(
    fecha DATE,
    variante VARCHAR(300),
    precio FLOAT,
    variacion FLOAT,
    id_producto INT,
    id_super INT,
    id_principal INT PRIMARY KEY,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_super) REFERENCES mercados(id_super) ON DELETE RESTRICT ON UPDATE CASCADE
);
""" 
cursor.execute(query_creacion)
conexion.commit()


In [25]:
lista_de_tuplas= []
for fila in df_principal.values:
    lista_de_tuplas.append(tuple(fila))

In [26]:
query_insercion= "INSERT INTO principal (fecha, variante, precio, variacion, id_producto, id_super, id_principal) VALUES (%s, %s, %s, %s, %s, %s,%s);"      #%s es como el format de los strings (la f)

cursor.executemany(query_insercion, lista_de_tuplas)                 #Executemany es para poder insertar varios valores
conexion.commit()

QUERYS PARA ANALIZAR

Comparación de Precios entre Supermercados: Determinar qué supermercados ofrecen los precios más bajos y cuáles son más caros para cada producto.

In [27]:
cursor=conexion.cursor()
query_precios_super="""
SELECT m.super ,p2.producto, MIN(p.precio) AS PrecioMin,  MAX(p.precio) AS PrecioMax
FROM mercados m
INNER JOIN principal p ON m.id_super = p.id_super
INNER JOIN productos p2 ON p.id_producto = p2.id_producto 
GROUP BY m.super, p2.producto 
ORDER BY PrecioMin, PrecioMax DESC;
"""
cursor.execute(query_precios_super)
df_precios_super_mi_max= pd.DataFrame(cursor.fetchall())

In [28]:
df_precios_super_mi_max.head()

,0,1,2,3
0,alcampo,leche,0.50,23.43
1,hipercor,leche,0.64,16.95
2,dia,leche,0.69,14.64
3,carrefour,leche,0.72,46.30
4,eroski,leche,0.73,6.85


Análisis de la Evolución de Precios: Estudiar cómo han cambiado los precios de los productos a lo largo del tiempo en distintos supermercados.

In [29]:
cursor=conexion.cursor()
query_evol_precios="""
SELECT p.fecha, m.super, p2.producto, SUM(p.variacion) OVER (PARTITION BY m.super, p2.producto ORDER BY p.fecha) AS PrecioAcumulado
FROM mercados m
INNER JOIN principal p ON m.id_super = p.id_super
INNER JOIN productos p2 ON p.id_producto = p2.id_producto
ORDER BY p2.producto, m.super, p.fecha;
"""
cursor.execute(query_evol_precios)
df_evol_precios_super= pd.DataFrame(cursor.fetchall())

In [30]:
df_evol_precios_super.sample(5)

,0,1,2,3
95255,2024-09-19,dia,leche,-0.50
5098,2024-07-30,hipercor,aceite-de-girasol,-0.18
29286,2024-08-06,dia,aceite-de-oliva,-18.99
2194,2024-08-19,carrefour,aceite-de-girasol,-2.36
55562,2024-09-10,hipercor,aceite-de-oliva,-206.77


Detección de Anomalías: Identificar subidas o bajadas de precios inusuales que podrían señalar prácticas abusivas o promociones.

In [31]:
cursor=conexion.cursor()
query_anomalia_precios="""
SELECT fecha, super, p2.producto, sum(p.variacion)  
FROM mercados m 
INNER JOIN principal p ON m.id_super = p.id_super
INNER JOIN productos p2 ON p.id_producto = p2.id_producto 
GROUP BY super, p2.producto, fecha;
"""
cursor.execute(query_anomalia_precios)
df_precios_anomalias_supers= pd.DataFrame(cursor.fetchall())

In [32]:
df_precios_anomalias_supers.sample(5)

,0,1,2,3
1514,2024-10-25,hipercor,aceite-de-oliva,-5.08
1390,2024-10-25,hipercor,aceite-de-girasol,0.00
1269,2024-09-28,eroski,leche,0.00
1003,2024-08-20,eroski,aceite-de-girasol,0.00
781,2024-08-09,dia,aceite-de-oliva,0.00


Análisis de la Dispersión de Precios: Evaluar la variabilidad de los precios de un mismo producto en diferentes supermercados.  

PARA HACER ESTO UN BOX PLOT DE LA VARIACION DEL PRECIO



Comparación de Precios Promedio: Calcular y comparar los precios promedio de cada producto en diferentes supermercados.

In [33]:
cursor=conexion.cursor()
query_precios_super="""
SELECT m.super ,p2.producto, p.precio
FROM mercados m
INNER JOIN principal p ON m.id_super = p.id_super
INNER JOIN productos p2 ON p.id_producto = p2.id_producto 
WHERE fecha = current_date ;
"""
cursor.execute(query_precios_super)
df_precios_super_mi_max= pd.DataFrame(cursor.fetchall())